In [2]:
import os
import sys
import datetime
import numpy as np
import pickle
import pandas as pd
import datetime
import re
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
main_file_path = r"Data\pir_Check.csv"
main_df = pd.read_csv(main_file_path, parse_dates=['airDate']).drop_duplicates()
main_df.head(3)

,url,episodeID,errorCode,footer,urlParam,EpsCategory,airDate
0,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29
10,https://tpirepguide.com/?p=18045,9781K,0,"March 28th, 2022 | Category: Carey Eps",18045,Carey Eps,2022-03-28
11,https://tpirepguide.com/?p=18018,9775K,0,"March 25th, 2022 | Category: Carey Eps",18018,Carey Eps,2022-03-25


In [3]:
main_df.shape

(6231, 7)

### Showcase Spins

In [160]:
sc_file_path = r"Data\pir_Content.csv"
sc_df = pd.read_csv(sc_file_path).drop_duplicates()
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4
0,1,"1,340",Maurice,85,0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN
1,2,"8,531",Kim,1,0,1,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN
2,3,"25,760",Sandra,20,50,70,NaN,C:,SS1,9782K,NaN,NaN,NaN


In [161]:
sc_df.shape

(26959, 13)

In [162]:
raw_df = sc_df.copy()

### Showcase HouseKeeping
- Three contestants for each showcase portion
- Add Contestant Order
- Check for "=" in Total Column

In [163]:
def keep_only_with_three(df):
    """Keep only those that have 3 contestants"""
    contestant_counts = df.groupby(['urlParam', 'Showcase']).SC_ind.count().reset_index()
    display(contestant_counts.SC_ind.value_counts(normalize=True))
    keeps = contestant_counts.loc[contestant_counts.SC_ind == 3].copy()
    df = df.merge(keeps, how='inner', on=['urlParam', 'Showcase'], suffixes=("", "_e"))
    del df['SC_ind_e']
    return df


In [164]:
sc_df = keep_only_with_three(sc_df)

3    0.937824
2    0.026518
6    0.025276
1    0.006319
4    0.002708
5    0.001354
Name: SC_ind, dtype: float64

In [165]:
print(sc_df.shape)
sc_df.head(3)

(24933, 13)


,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4
0,1,"1,340",Maurice,85,0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN
1,2,"8,531",Kim,1,0,1,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN
2,3,"25,760",Sandra,20,50,70,NaN,C:,SS1,9782K,NaN,NaN,NaN


#### Create Contestant Order Column

In [166]:
sc_df['ContestantOrder'] = sc_df.groupby(['urlParam', 'Showcase'])['SC_ind'].rank()
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder
0,1,"1,340",Maurice,85,0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN,1.0
1,2,"8,531",Kim,1,0,1,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN,2.0
2,3,"25,760",Sandra,20,50,70,NaN,C:,SS1,9782K,NaN,NaN,NaN,3.0


In [167]:
sc_df.ContestantOrder.value_counts()

2.0    8311
3.0    8311
1.0    8311
Name: ContestantOrder, dtype: int64

#### Clean Up Comments

In [168]:
sc_df['CommNums'] = sc_df.Comments.apply(lambda x: re.findall(r'[\d]*\.[\d]+', x))

EV3 Populated

In [169]:
sc_df.loc[(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev3.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums
17915,4,"5,340",Jeffrey,0.55,0.05,=,35.0,C: 0.60 Jeffrey FINALLY goes to Showcase!!,SS1,0042K,60.0,45.0,NaN,3.0,[0.60]


In [170]:
# In above the 0.60 is the total
sc_df['Total'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev3.isna())],
                          [sc_df.CommNums.apply(lambda x: x[0] if len(x) > 0 else None)], sc_df.Total)

In [171]:
sc_df['Fixed'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev3.isna()),
                           (sc_df.CommNums.apply(lambda x: len(x)) < 1) & (sc_df.Total != "=")], [True, True], False)

EV2 Populated

In [172]:
sc_df.loc[(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev2.isna()) & ~sc_df.Fixed]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Fixed
2345,4,"12,866",Marilyn,80,0,80,90.0,C: 1.00 Bonus Spin ►,SS2,8911K,25.0,NaN,NaN,3.0,[1.00],False
15733,3,"1,574+",Donald,25,65,=,90.0,C: 1.00 BONUS,SS2,1392K,90.0,NaN,NaN,2.0,[1.00],False
22075,3,"1,049",Jan,55,25,80,55.0,C: 1.00 Bonus Spin >,SS1,5265D,20.0,NaN,NaN,2.0,[1.00],False
22823,4,"5,141",Stephen,50,0,50,45.0,C: 1.00 Bonus Spin >,SS1,4993D,60.0,NaN,NaN,3.0,[1.00],False


In [173]:
# Fix these, if = ev1 is total and CommNums needs to go to ev1; Else comm needs to go to ev2 and ev2 to 3
sc_df['WorkingOn'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev2.isna()) & (~sc_df.Fixed)],
                              [True], False)
sc_df['ev3'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')], [sc_df.ev2], sc_df.ev3)
sc_df['ev2'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')],
                         [sc_df.CommNums.apply(lambda x: x[0] if len(x) > 0 else None)], sc_df.ev2)
# Update working on to those with "=" in total
sc_df['WorkingOn'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != "="), sc_df.WorkingOn], [False, True], False)
sc_df['Total'] = np.select([(sc_df.WorkingOn)], [sc_df.ev1], sc_df.Total)
sc_df['ev1'] = np.select([(sc_df.WorkingOn)], [sc_df.CommNums.apply(lambda x: x[0] if len(x) > 0 else None)], sc_df.ev1)

In [174]:
sc_df['Fixed'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev2.isna()) & ~sc_df.Fixed], [True],
                           sc_df.Fixed)

EV1 Populated

In [175]:
sc_df[(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev1.isna()) & ~sc_df.Fixed & (sc_df.Total == "=")].head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Fixed,WorkingOn
16495,2,"1,500",Patricia,25,75,=,10.0,C: 1.00 BONUS:,SS1,1072K,NaN,NaN,NaN,2.0,[1.00],False,False
16608,1,950,Ryan,40,60,=,65.0,C: 1.00 to the showcase,SSBSO,1015K,NaN,NaN,NaN,1.0,[1.00],False,False
16609,2,"1,499",Marni,55,45,=,55.0,C: 1.00,SSBSO,1015K,NaN,NaN,NaN,2.0,[1.00],False,False


In [176]:
sc_df[(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev1.isna()) & ~sc_df.Fixed & (sc_df.Total != "=")].head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Fixed,WorkingOn
349,3,"13,086",William,65,20,85,40.0,C: 1.00 Bonus Spin >,SS1,9661K,NaN,NaN,NaN,2.0,[1.00],False,False
591,2,530,Kelly,65,0,65,15.0,C: 1.00 Bonus Spin ► DQ,SS1,9563K,NaN,NaN,NaN,1.0,[1.00],False,False
1065,2,"8,394",Kevin,25,65,90,90.0,C: 1.00 Bpnus Spin ►,SS1,9405K,NaN,NaN,NaN,1.0,[1.00],False,False


In [177]:
sc_df['WorkingOn'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev1.isna()) & (~sc_df.Fixed)],
                              [True], False)
sc_df['ev2'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')], [sc_df.ev1], sc_df.ev2)
sc_df['ev1'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')],
                         [sc_df.CommNums.apply(lambda x: x[0] if len(x) > 0 else None)], sc_df.ev1)
# Update working on to those with "=" in total
sc_df['WorkingOn'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != "="), sc_df.WorkingOn], [False, True], False)
sc_df['Total'] = np.select([(sc_df.WorkingOn)],  [sc_df.CommNums.apply(lambda x: x[0] if len(x) > 0 else None)],
                           sc_df.Total)

In [178]:
sc_df['Fixed'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (~sc_df.ev1.isna()) & ~sc_df.Fixed], [True],
                           sc_df.Fixed)

No Extra Values Populated

In [180]:
sc_df.loc[(sc_df.CommNums.apply(lambda x: len(x)) > 1) & (sc_df.ev1.isna()) & ~sc_df.Fixed]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Fixed,WorkingOn
1260,2,"1,499",Robert,85,0,85,NaN,"C: 1.00 Bonus Spin ► 1.00 25,000 Bonus",SS1,9333K,NaN,NaN,NaN,1.0,"[1.00, 1.00]",False,False
2273,4,"1,799",Anthony,60,10,70,NaN,"C: 1.00 1.00 25,000 Bonus",SS1,8942K,NaN,NaN,NaN,3.0,"[1.00, 1.00]",False,False
15707,4,"14,360",Ralph:,0.5,0.25,=,NaN,C: 0.75 1.00 Bonus > 0.90,SS1:,1401K,NaN,NaN,NaN,3.0,"[0.75, 1.00, 0.90]",False,False
16477,2,"1,130",Sammy,5,95,=,NaN,"C: 1.00 BONUS 1.00 *10,000*",SS1,1081K,NaN,NaN,NaN,2.0,"[1.00, 1.00]",False,False
17918,3,"20,001",Renee,0.7,0.3,=,NaN,C: 1.00 Bonus Spin = 0.25,SS2,0042K,NaN,NaN,NaN,3.0,"[1.00, 0.25]",False,False
19331,4,"3,811",Joan,90,0,90,NaN,"C: 1.00 Bonus Spin > 1.00 10,000 Bonus",SS1,8793D,NaN,NaN,NaN,3.0,"[1.00, 1.00]",False,False
23793,1,730,Aniteria,0.35,0.65,=,NaN,C: 1.00 Bonus Spin - 0.70,SS2:,4641D,NaN,NaN,NaN,1.0,"[1.00, 0.70]",False,False


In [182]:
sc_df['WorkingOn'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & (sc_df.ev1.isna()) & (~sc_df.Fixed)],
                              [True], False)
sc_df['ev1'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')],
                         [sc_df.CommNums.apply(lambda x: x[0] if len(x) > 0 else None)], sc_df.ev1)
sc_df['ev2'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')],
                         [sc_df.CommNums.apply(lambda x: x[1] if len(x) > 1 else None)], sc_df.ev2)
# Update working on to those with "=" in total
sc_df['WorkingOn'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != "="), sc_df.WorkingOn], [False, True], False)
sc_df['Total'] = np.select([(sc_df.WorkingOn)],  [sc_df.CommNums.apply(lambda x: x[0] if len(x) > 0 else None)],
                           sc_df.Total)
sc_df['ev1'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')],
                         [sc_df.CommNums.apply(lambda x: x[1] if len(x) > 1 else None)], sc_df.ev1)
sc_df['ev2'] = np.select([(sc_df.WorkingOn) & (sc_df.Total != '=')],
                         [sc_df.CommNums.apply(lambda x: x[2] if len(x) > 2 else None)], sc_df.ev2)

In [183]:
sc_df['Fixed'] = np.select([(sc_df.CommNums.apply(lambda x: len(x)) > 0) & ~sc_df.Fixed], [True],
                           sc_df.Fixed)

Those with no values in the comments but an Equals in the Total

In [185]:
sc_df.loc[~sc_df.Fixed].shape, sc_df.loc[~sc_df.Fixed & (sc_df.Total == "=")].shape

((1604, 17), (1604, 17))

In [198]:
sc_df.loc[~sc_df.Fixed & ~sc_df.ev4.isna()]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Fixed,WorkingOn
15453,2,"1,770",Christopher,45,45,=,90.0,C: bonus spin,SS1,1511K,55.0,100.0,95.0,1.0,[],False,False
16540,3,"1,645",Daniel,5,80,=,85.0,C: Through to the Showcases,SS2,1051K,10.0,50.0,95.0,2.0,[],False,False
16541,4,"4,025",Jaime,50,35,=,85.0,C:,SS2,1051K,10.0,50.0,10.0,3.0,[],False,False


In [199]:
# Move everything forward
sc_df['Total'] = np.select([~sc_df.Fixed], [sc_df.ev1], sc_df.Total)
sc_df['ev1'] = np.select([~sc_df.Fixed], [sc_df.ev2], sc_df.ev1)
sc_df['ev2'] = np.select([~sc_df.Fixed], [sc_df.ev3], sc_df.ev2)
sc_df['ev3'] = np.select([~sc_df.Fixed], [sc_df.ev4], sc_df.ev3)
sc_df['ev4'] = np.select([~sc_df.Fixed], [None], sc_df.ev4)

In [203]:
# All done
del sc_df['Fixed']
del sc_df['WorkingOn']

In [204]:
new_base = sc_df.copy()

### Confirm Numbers in Spin1, Spin2, and Total

In [222]:
def make_numeric_column(df, col):
    df[col + '_test'] = df[col].astype(str).str.replace(r'[\.,$\s\+\*-]', '', regex=True)
    # replace empty string with 0
    df[col + '_test'] = np.select([df[col + '_test'] == ''], [0], df[col + '_test'])
    num_numeric = sum(df[col + '_test'].apply(lambda x: str(x).isnumeric()))
    if num_numeric == len(df):
        df[col] = df[col + '_test']
        del df[col + '_test']
    return df


In [228]:
sc_df = make_numeric_column(sc_df, 'Spin1')
sc_df = make_numeric_column(sc_df, 'Spin2')

In [229]:
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Spin2_test
0,1,"1,340",Maurice,85,0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN,1.0,[],0
1,2,"8,531",Kim,1,0,1,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN,2.0,[],0
2,3,"25,760",Sandra,20,50,70,NaN,C:,SS1,9782K,NaN,NaN,NaN,3.0,[],50


In [230]:
sc_df.loc[~sc_df.Spin2_test.apply(lambda x: str(x).isnumeric())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Spin2_test
17309,4,"1,539",Caralisa,15,(Boo),45,60.0,C: =,SS2,0495K,NaN,NaN,NaN,3.0,[],(Boo)
17694,1,560,Kristina,5,(Wheelie),5,10.0,C: =,SS1,0243K,NaN,NaN,NaN,1.0,[],(Wheelie)


In [231]:
# Would just drop if writing a script, but will fix here as easy enough
sc_df['fixMe'] = ~sc_df['Spin2_test'].apply(lambda x: str(x).isnumeric())
sc_df['Spin2'] = np.select([sc_df.fixMe], [sc_df.Total], sc_df['Spin2_test'])
sc_df['Total'] = np.select([sc_df.fixMe], [sc_df.ev1], sc_df['Total'])
sc_df['Comments'] = np.select([sc_df.fixMe], [sc_df['Comments'] + sc_df.Spin2_test.astype(str)], sc_df['Comments'])
sc_df.loc[sc_df.fixMe]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Spin2_test,fixMe
17309,4,"1,539",Caralisa,15,45,60.0,60.0,C: =(Boo),SS2,0495K,NaN,NaN,NaN,3.0,[],(Boo),True
17694,1,560,Kristina,5,5,10.0,10.0,C: =(Wheelie),SS1,0243K,NaN,NaN,NaN,1.0,[],(Wheelie),True


In [232]:
del sc_df['Spin2_test']
del sc_df['fixMe']

In [234]:
sc_df['Spin1'] = sc_df['Spin1'].astype(np.int64)
sc_df['Spin2'] = sc_df['Spin2'].astype(np.int64)
sc_df = make_numeric_column(sc_df, 'Total')
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
0,1,"1,340",Maurice,85,0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN,1.0,[],85
1,2,"8,531",Kim,1,0,1,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN,2.0,[],1
2,3,"25,760",Sandra,20,50,70,NaN,C:,SS1,9782K,NaN,NaN,NaN,3.0,[],70


In [237]:
sc_df['fixMe'] = ~sc_df['Total_test'].apply(lambda x: str(x).isnumeric())
sc_df.loc[sc_df.fixMe].Total_test.value_counts()

OVER       322
OV         214
nan         12
Through      4
Bonus        3
bonus        1
Name: Total_test, dtype: int64

#### Clean up values if not between 5 and 100 or multiple of 5

In [268]:
sc_df.loc[(sc_df.Spin1 < 5) | ((sc_df.Spin2 < 5) & (sc_df.Spin2 > 0))]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
1,2,"8,531",Kim,1,0,1,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN,2.0,[],1
9,1,750,Ruby,1,0,1,80.0,C: Bonus Spin >,SS2,9781K,NaN,NaN,NaN,1.0,[],1
11,3,"9,168",Rachel,70,1,71,NaN,C:,SS2,9781K,NaN,NaN,NaN,3.0,[],71
20,3,"3,138",Lorrie,40,1,41,NaN,C:,SS1,9774K,NaN,NaN,NaN,3.0,[],41
29,3,"3,089",Lucille,10,1,11,NaN,C:,SS2,9773K,NaN,NaN,NaN,3.0,[],11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21752,3,"7,786",Cheryl,1,0,1,1.00,"C: Bonus Spin > 1.00 10,000 Bonus",SS2,5384D,None,NaN,NaN,3.0,[1.00],1
21761,3,"3,220",Joyce,1,0,1,30.0,C: Bonus Spin >,SS1,5382D,NaN,NaN,NaN,3.0,[],1
21765,1,365,Jean,1,0,1,20.0,C: Bonus Spin >,SS1,5381D,NaN,NaN,NaN,1.0,[],1
21780,1,924,Lynne,45,1,46,NaN,C:,SS2,5374D,NaN,NaN,NaN,1.0,[],46


In [269]:
sc_df.loc[(sc_df.Spin1 > 100) | (sc_df.Spin2 > 100)]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test


In [270]:
sc_df['Spin1'] = np.select([sc_df.Spin1 < 5], [sc_df.Spin1 * 100], sc_df.Spin1)
sc_df['Spin2'] = np.select([sc_df.Spin2 < 5], [sc_df.Spin2 * 100], sc_df.Spin2)
sc_df['Spin1'] = sc_df['Spin1'].astype(np.int64)
sc_df['Spin2'] = sc_df['Spin2'].astype(np.int64)
sc_df.loc[(sc_df.Spin1 > 100) | (sc_df.Spin2 > 100)]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
15709,2,"5,668",Tamera:,300,45,48,None,C: 0.75 advances to the Showcase Round,SS2:,1401K,None,NaN,NaN,2.0,[0.75],075
16500,1,"1,070",Yolanda:,15,400,19,NaN,C: Through to the Showcases,SS1:,1071K,NaN,NaN,NaN,1.0,[],055
17494,2,"5,319",Angela,200,0,2,15.0,C: + =,SS1,0355K,20.0,45.0,NaN,2.0,[],2
17916,1,"1,067",James,300,6,9,None,C: 0.90,SS2,0042K,None,NaN,NaN,1.0,[0.90],090
17918,3,"20,001",Renee,7,300,10,0.25,C: 1.00 Bonus Spin = 0.25,SS2,0042K,None,NaN,NaN,3.0,"[1.00, 0.25]",100


In [274]:
sc_df.loc[(sc_df.Spin1 % 5 != 0) | (sc_df.Spin2 % 5 != 0)]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
10990,2,"8,892",Amy,55,57,112,NaN,C:,SS1,5614K,NaN,NaN,NaN,2.0,[],112
11277,1,"2,339",Christina,50,38,88,NaN,C:,SS2,5513K,NaN,NaN,NaN,1.0,[],85
14677,2,"6,299",Karen,50,38,88,NaN,C: Through to the showcases,SS1,4263K,NaN,NaN,NaN,2.0,[],85
16505,4,"17,626",Leo:,85,8,93,NaN,C:,SS2:,1071K,NaN,NaN,NaN,3.0,[],165
17914,3,"5,196",Tyler,6,0,6,35.0,C:,SS1,0042K,60.0,25.0,NaN,2.0,[],06
17916,1,"1,067",James,300,6,9,None,C: 0.90,SS2,0042K,None,NaN,NaN,1.0,[0.90],090
17918,3,"20,001",Renee,7,300,10,0.25,C: 1.00 Bonus Spin = 0.25,SS2,0042K,None,NaN,NaN,3.0,"[1.00, 0.25]",100


In [275]:
# One last fix
sc_df['FixMe'] = np.select([(sc_df.Spin1 > 100) | (sc_df.Spin2 > 100) | (sc_df.Spin1 % 5 != 0) | (sc_df.Spin2 % 5 != 0)],
                           [True], False)
sc_df[sc_df.FixMe]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,FixMe
10990,2,"8,892",Amy,55,57,112,NaN,C:,SS1,5614K,NaN,NaN,NaN,2.0,[],112,True
11277,1,"2,339",Christina,50,38,88,NaN,C:,SS2,5513K,NaN,NaN,NaN,1.0,[],85,True
14677,2,"6,299",Karen,50,38,88,NaN,C: Through to the showcases,SS1,4263K,NaN,NaN,NaN,2.0,[],85,True
15709,2,"5,668",Tamera:,300,45,48,None,C: 0.75 advances to the Showcase Round,SS2:,1401K,None,NaN,NaN,2.0,[0.75],075,True
16500,1,"1,070",Yolanda:,15,400,19,NaN,C: Through to the Showcases,SS1:,1071K,NaN,NaN,NaN,1.0,[],055,True
16505,4,"17,626",Leo:,85,8,93,NaN,C:,SS2:,1071K,NaN,NaN,NaN,3.0,[],165,True
17494,2,"5,319",Angela,200,0,2,15.0,C: + =,SS1,0355K,20.0,45.0,NaN,2.0,[],2,True
17914,3,"5,196",Tyler,6,0,6,35.0,C:,SS1,0042K,60.0,25.0,NaN,2.0,[],06,True
17916,1,"1,067",James,300,6,9,None,C: 0.90,SS2,0042K,None,NaN,NaN,1.0,[0.90],090,True
17918,3,"20,001",Renee,7,300,10,0.25,C: 1.00 Bonus Spin = 0.25,SS2,0042K,None,NaN,NaN,3.0,"[1.00, 0.25]",100,True


In [276]:
sc_df['Spin1'] = np.select([sc_df.FixMe & (sc_df.Spin1 > 100), sc_df.FixMe & (sc_df.Spin1 < 10)],
                          [sc_df.Spin1 / 10, sc_df.Spin1 * 10], sc_df.Spin1)
sc_df['Spin2'] = np.select([sc_df.FixMe & (sc_df.Spin2 > 100), sc_df.FixMe & (sc_df.Spin2 < 10)],
                          [sc_df.Spin2 / 10, sc_df.Spin2 * 10], sc_df.Spin2)
sc_df['FixMe'] = np.select([(sc_df.Spin1 > 100) | (sc_df.Spin2 > 100) | (sc_df.Spin1 % 5 != 0) | (sc_df.Spin2 % 5 != 0)],
                           [True], False)
sc_df[sc_df.FixMe]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,FixMe
10990,2,"8,892",Amy,55.0,57.0,112,NaN,C:,SS1,5614K,NaN,NaN,NaN,2.0,[],112,True
11277,1,"2,339",Christina,50.0,38.0,88,NaN,C:,SS2,5513K,NaN,NaN,NaN,1.0,[],85,True
14677,2,"6,299",Karen,50.0,38.0,88,NaN,C: Through to the showcases,SS1,4263K,NaN,NaN,NaN,2.0,[],85,True


In [281]:
# Assume typo - round Down
sc_df['Spin2'] = sc_df['Spin2'] // 5 * 5

In [285]:
# Just going to make the total correct assuming Spin1 and Spin2 are accurate
sc_df.Total = sc_df.Spin1 + sc_df.Spin2
sc_df.Total = sc_df.Total.astype(np.int64)
del sc_df['FixMe']
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
0,1,"1,340",Maurice,85.0,0.0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN,1.0,[],85
1,2,"8,531",Kim,100.0,0.0,100,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN,2.0,[],1
2,3,"25,760",Sandra,20.0,50.0,70,NaN,C:,SS1,9782K,NaN,NaN,NaN,3.0,[],70


### Find SpinOff Extra Values

In [286]:
base_df = sc_df.copy()

In [417]:
sc_df = base_df.copy()

#### First Fix Extra values that may  contain bonus amounts

In [418]:
sc_df['ev1'] = sc_df.ev1.astype(float)
sc_df['ev2'] = sc_df.ev2.astype(float)
sc_df['ev3'] = sc_df.ev3.astype(float)
sc_df['ev4'] = sc_df.ev4.astype(float)

In [419]:
sc_df.loc[(sc_df.ev4 > 100) | (sc_df.ev4 < 1) | ((sc_df.ev4 % 5 != 0) & ~sc_df.ev4.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test


In [420]:
sc_df.loc[(sc_df.ev3 > 100) | (sc_df.ev3 < 1) | ((sc_df.ev3 % 5 != 0) & ~sc_df.ev3.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test


In [421]:
sc_df.loc[(sc_df.ev2 > 100) | (sc_df.ev2 < 1) | ((sc_df.ev2 % 5 != 0) & ~sc_df.ev2.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
1260,2,"1,499",Robert,85.0,0.0,85,1.0,"C: 1.00 Bonus Spin ► 1.00 25,000 Bonus",SS1,9333K,1.0,NaN,NaN,1.0,"[1.00, 1.00]",85
2273,4,"1,799",Anthony,60.0,10.0,70,1.0,"C: 1.00 1.00 25,000 Bonus",SS1,8942K,1.0,NaN,NaN,3.0,"[1.00, 1.00]",70
2345,4,"12,866",Marilyn,80.0,0.0,80,90.0,C: 1.00 Bonus Spin ►,SS2,8911K,1.0,25.0,NaN,3.0,[1.00],80
15707,4,"14,360",Ralph:,5.0,25.0,30,1.0,C: 0.75 1.00 Bonus > 0.90,SS1:,1401K,0.9,NaN,NaN,3.0,"[0.75, 1.00, 0.90]",075
17040,1,"4,598",Todd,50.0,50.0,100,5.0,C: Bonus,SSB/SO,0744K,5000.0,NaN,NaN,1.0,[],1
17043,1,"4,738",Milton,100.0,0.0,100,15.0,C: Bonus bonus,SSSHOWDOWN:,0744K,5000.0,NaN,NaN,1.0,[],1
17881,2,"7,337",Alicia,100.0,0.0,100,15.0,C: Bonus,SSB/SO,0091K,5000.0,NaN,NaN,2.0,[],1
19331,4,"3,811",Joan,90.0,0.0,90,1.0,"C: 1.00 Bonus Spin > 1.00 10,000 Bonus",SS1,8793D,1.0,NaN,NaN,3.0,"[1.00, 1.00]",90
22075,3,"1,049",Jan,55.0,25.0,80,55.0,C: 1.00 Bonus Spin >,SS1,5265D,1.0,20.0,NaN,2.0,[1.00],80
22823,4,"5,141",Stephen,50.0,0.0,50,45.0,C: 1.00 Bonus Spin >,SS1,4993D,1.0,60.0,NaN,3.0,[1.00],50


In [422]:
sc_df.loc[(sc_df.ev1 > 100) | (sc_df.ev1 < 1) | ((sc_df.ev1 % 5 != 0) & ~sc_df.ev1.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
342,1,"5,232",Kristen,100.0,0.0,100,1.0,"C: Bonus Spin > 1.00 25,000 Bonus",SS1,9662K,NaN,NaN,NaN,1.0,[1.00],1
349,3,"13,086",William,65.0,20.0,85,1.0,C: 1.00 Bonus Spin >,SS1,9661K,40.0,NaN,NaN,2.0,[1.00],85
369,2,"1,400",Theresa,100.0,0.0,100,1.0,"C: 1.00 25,000 Bonus",SS2,9653K,NaN,NaN,NaN,1.0,[1.00],1
384,2,"7,081",Peter,100.0,0.0,100,1.0,"C: 1.00 25,000 Bonus",SS1,9645K,NaN,NaN,NaN,1.0,[1.00],1
575,3,"12,979",Kenya,50.0,50.0,100,1.0,"C: Bonus Spin ► 1.00 *25,000*",SS1,9591K,NaN,NaN,NaN,3.0,[1.00],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24187,3,"3,660",Miriam,40.0,30.0,70,1.0,C: 1.00 Bonus Spin >>,SS2,4164D,40.0,NaN,NaN,2.0,[1.00],70
24367,3,"3,660",Miriam,40.0,30.0,70,1.0,C: 1.00 Bonus Spin >,SS2,3721D,40.0,NaN,NaN,2.0,[1.00],70
24494,3,"5,334",Cindy,100.0,0.0,100,1.0,"C: Bonus Spin 1.00 10,000 Bonus",SS1,3444D,NaN,NaN,NaN,3.0,[1.00],100
24925,3,858,Luis,15.0,85.0,100,1000.0,C: to the showcase,SS2,1581D,95.0,NaN,NaN,2.0,[],100


In [423]:
sc_df['ev2'] = np.select([sc_df.ev2 > 100, sc_df.ev2 <= 1], [None, sc_df.ev2 * 100], sc_df.ev2)
# First those where 1000 for first dollar in ev1 fix
sc_df['FixMe'] = np.select([sc_df.ev1 > 100], [True], False)
sc_df['ev1'] = np.select([sc_df.FixMe], [sc_df.ev2], sc_df.ev1)
sc_df['ev2'] = np.select([sc_df.FixMe], [None], sc_df.ev2)
sc_df['ev1'] = np.select([sc_df.ev1 <= 1], [sc_df.ev1 * 100], sc_df.ev1)

In [424]:
del sc_df['FixMe']
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
0,1,"1,340",Maurice,85.0,0.0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN,1.0,[],85
1,2,"8,531",Kim,100.0,0.0,100,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN,2.0,[],1
2,3,"25,760",Sandra,20.0,50.0,70,NaN,C:,SS1,9782K,NaN,NaN,NaN,3.0,[],70


In [425]:
sc_df.loc[(sc_df.ev1 > 100) | (sc_df.ev1 < 5) | ((sc_df.ev1 % 5 != 0) & ~sc_df.ev1.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test
13494,1,700,William,95.0,0.0,95,2.0,C: Thru teh showcasez,SS2,4675K,NaN,NaN,NaN,1.0,[],95
18738,1,572,Olivia,60.0,65.0,125,2.0,C: (puny 2nd spins),SS2,9473D,NaN,NaN,NaN,1.0,[],OV


In [426]:
sc_df.loc[(sc_df.ev2 > 100) | (sc_df.ev2 < 1) | ((sc_df.ev2 % 5 != 0) & ~sc_df.ev2.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test


In [427]:
sc_df['ev1'] = np.select([sc_df.ev1 < 5], [None], sc_df.ev1)

In [428]:
sc_df.loc[(sc_df.ev1 > 100) | (sc_df.ev1 < 5) | ((sc_df.ev1 % 5 != 0) & ~sc_df.ev1.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test


#### Find the First SpinOffs

In [429]:
def get_tie_df(df, col, group_cols=['urlParam', 'Showcase']):
    """Find those with the same max value in specified column"""
    sc_maxes = df.loc[~df[col].isna(), :].groupby(group_cols)[col].max().reset_index()
    new_df = df.merge(sc_maxes, how='left', on=group_cols, suffixes=("", "Max"))
    new_df[col + 'Match'] = (new_df[col + 'Max'] == df[col])
    # Count the number of those Matching the "Max"
    sc_tie_cnt = new_df.groupby(group_cols)[col + 'Match'].sum().reset_index()
    new_df = new_df.merge(sc_tie_cnt, how='left', on=group_cols, suffixes=("", "Cnt"))
    return new_df


In [430]:
# Find ties first
sc_df['EndVal'] = np.select([sc_df.Total > 100], [0], sc_df.Total)
sc_df = get_tie_df(sc_df, 'EndVal')
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,EndVal,EndValMax,EndValMatch,EndValMatchCnt
0,1,"1,340",Maurice,85.0,0.0,85,NaN,C:,SS1,9782K,NaN,NaN,NaN,1.0,[],85,85,100,False,1
1,2,"8,531",Kim,100.0,0.0,100,50.0,C: Bonus Spin ►,SS1,9782K,NaN,NaN,NaN,2.0,[],1,100,100,True,1
2,3,"25,760",Sandra,20.0,50.0,70,NaN,C:,SS1,9782K,NaN,NaN,NaN,3.0,[],70,70,100,False,1


In [431]:
print("Showcases w/o SpinOff Value: ", 
      sc_df.loc[(sc_df.EndValMatchCnt > 1) & sc_df.EndValMatch & (sc_df.ev1.isna())].urlParam.nunique())
sc_df.loc[(sc_df.EndValMatchCnt > 1) & sc_df.EndValMatch & (sc_df.ev1.isna())].head(5)

Showcases w/o SpinOff Value:  29


,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,EndVal,EndValMax,EndValMatch,EndValMatchCnt
2788,3,"1,858",Giovanni,45.0,55.0,100,NaN,C: Through to the Showcases,SS2,8674K,NaN,NaN,NaN,2.0,[],65,100,100,True,2
3418,2,988,Robbie,25.0,65.0,90,NaN,C: Through to the showcase,SS2,8494K,NaN,NaN,NaN,2.0,[],90,90,90,True,2
3419,3,"2,309",Micah,15.0,75.0,90,NaN,C:,SS2,8494K,NaN,NaN,NaN,3.0,[],85,90,90,True,2
4255,3,"6,810",Kathryn,5.0,90.0,95,NaN,C: Through to the showcases,SS2,8142K,NaN,NaN,NaN,2.0,[],85,95,95,True,2
4597,3,"1,960",Shasta,85.0,0.0,85,NaN,C:,SS1,8012K,NaN,NaN,NaN,2.0,[],85,85,85,True,2


In [432]:
# Will drop the have no value to extract
sc_df['SpinOff1'] = np.select([(sc_df.EndValMatch) & (sc_df.EndValMatchCnt > 1) & ~sc_df.ev1.isna()], [sc_df.ev1], None)
sc_df['SpinOff1'] = sc_df['SpinOff1'].astype(float)
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,...,ev3,ev4,ContestantOrder,CommNums,Total_test,EndVal,EndValMax,EndValMatch,EndValMatchCnt,SpinOff1
0,1,"1,340",Maurice,85.0,0.0,85,NaN,C:,SS1,9782K,...,NaN,NaN,1.0,[],85,85,100,False,1,NaN
1,2,"8,531",Kim,100.0,0.0,100,50.0,C: Bonus Spin ►,SS1,9782K,...,NaN,NaN,2.0,[],1,100,100,True,1,NaN
2,3,"25,760",Sandra,20.0,50.0,70,NaN,C:,SS1,9782K,...,NaN,NaN,3.0,[],70,70,100,False,1,NaN


In [433]:
sc_df['Removal'] = np.select([(sc_df.EndValMatch) & (sc_df.EndValMatchCnt > 1) & sc_df.SpinOff1.isna()], [True], False)
sc_df[sc_df.Removal]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,...,ev4,ContestantOrder,CommNums,Total_test,EndVal,EndValMax,EndValMatch,EndValMatchCnt,SpinOff1,Removal
2788,3,"1,858",Giovanni,45.0,55.0,100,NaN,C: Through to the Showcases,SS2,8674K,...,NaN,2.0,[],65,100,100,True,2,NaN,True
3418,2,988,Robbie,25.0,65.0,90,NaN,C: Through to the showcase,SS2,8494K,...,NaN,2.0,[],90,90,90,True,2,NaN,True
3419,3,"2,309",Micah,15.0,75.0,90,NaN,C:,SS2,8494K,...,NaN,3.0,[],85,90,90,True,2,NaN,True
4255,3,"6,810",Kathryn,5.0,90.0,95,NaN,C: Through to the showcases,SS2,8142K,...,NaN,2.0,[],85,95,95,True,2,NaN,True
4597,3,"1,960",Shasta,85.0,0.0,85,NaN,C:,SS1,8012K,...,NaN,2.0,[],85,85,85,True,2,NaN,True
5797,3,"7,622",Walter,55.0,5.0,60,NaN,C:,SS2,7552K,...,NaN,2.0,[],70,60,60,True,2,NaN,True
10899,2,668,Brett,100.0,0.0,100,NaN,C: 60* Through to the Showcases,SS1,5682K,...,NaN,1.0,[],1,100,100,True,2,NaN,True
11673,2,"1,620",Thu,40.0,55.0,95,NaN,C:,SS2,5351K,...,NaN,1.0,[],5,95,95,True,2,NaN,True
11674,3,"3,691",Justin,15.0,80.0,95,NaN,C: Through to the Showcases,SS2,5351K,...,NaN,2.0,[],95,95,95,True,2,NaN,True
12800,4,"19,373",Luke,30.0,65.0,95,NaN,C:,SS2,4913K,...,NaN,3.0,[],50,95,95,True,2,NaN,True


#### 2nd Spinoffs

In [434]:
sc_df = get_tie_df(sc_df, 'SpinOff1')
sc_df.loc[sc_df.SpinOff1MatchCnt > 1]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,...,Total_test,EndVal,EndValMax,EndValMatch,EndValMatchCnt,SpinOff1,Removal,SpinOff1Max,SpinOff1Match,SpinOff1MatchCnt
45,2,627,Vanessa,10.0,85.0,95,95.0,C: Through to the Showcases,SS2,9763K,...,95,95,95,True,3,95.0,False,95.0,True,2
46,3,"2,486",Reynaldo,95.0,0.0,95,75.0,C:,SS2,9763K,...,95,95,95,True,3,75.0,False,95.0,False,2
47,4,"16,680",Ayesha,95.0,0.0,95,95.0,C:,SS2,9763K,...,95,95,95,True,3,95.0,False,95.0,True,2
729,2,"9,567",Joven,20.0,65.0,85,85.0,C: Through to the Showcases,SS2,9531K,...,85,85,85,True,2,85.0,False,85.0,True,2
730,3,"12,623",Kirk,65.0,50.0,115,NaN,C:,SS2,9531K,...,115,0,85,False,2,NaN,False,85.0,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23839,3,"2,966",Janet,40.0,65.0,105,NaN,C:,SS2,4622D,...,1050,0,80,False,2,NaN,False,90.0,False,2
23840,4,"7,702",Mark,5.0,75.0,80,90.0,C:,SS2,4622D,...,80,80,80,True,2,90.0,False,90.0,True,2
24237,2,930,Kathleen,75.0,0.0,75,NaN,C:,SS1,3954D,...,75,75,90,False,2,NaN,False,85.0,False,2
24238,3,"3,270",Randy,90.0,0.0,90,85.0,C: Through to the Showcases,SS1,3954D,...,90,90,90,True,2,85.0,False,85.0,True,2


In [435]:
sc_df.loc[sc_df.SpinOff1MatchCnt > 1, :].urlParam.nunique()

56

In [436]:
sc_df.loc[(sc_df.SpinOff1MatchCnt > 1) & (sc_df.SpinOff1Match)]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,...,Total_test,EndVal,EndValMax,EndValMatch,EndValMatchCnt,SpinOff1,Removal,SpinOff1Max,SpinOff1Match,SpinOff1MatchCnt
45,2,627,Vanessa,10.0,85.0,95,95.0,C: Through to the Showcases,SS2,9763K,...,95,95,95,True,3,95.0,False,95.0,True,2
47,4,"16,680",Ayesha,95.0,0.0,95,95.0,C:,SS2,9763K,...,95,95,95,True,3,95.0,False,95.0,True,2
729,2,"9,567",Joven,20.0,65.0,85,85.0,C: Through to the Showcases,SS2,9531K,...,85,85,85,True,2,85.0,False,85.0,True,2
731,4,"19,496",Kayla,40.0,45.0,85,85.0,C:,SS2,9531K,...,85,85,85,True,2,85.0,False,85.0,True,2
853,3,"2,400",Gustave,95.0,0.0,95,20.0,C:,SS2,9482K,...,95,95,95,True,2,20.0,False,20.0,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23399,4,"3,016",Beverly,90.0,0.0,90,75.0,C: Through to the Showcases,SS2,4783D,...,90,90,90,True,2,75.0,False,75.0,True,2
23838,2,350,Patricia,55.0,25.0,80,90.0,C: Through to the Showcases,SS2,4622D,...,80,80,80,True,2,90.0,False,90.0,True,2
23840,4,"7,702",Mark,5.0,75.0,80,90.0,C:,SS2,4622D,...,80,80,80,True,2,90.0,False,90.0,True,2
24238,3,"3,270",Randy,90.0,0.0,90,85.0,C: Through to the Showcases,SS1,3954D,...,90,90,90,True,2,85.0,False,85.0,True,2


In [437]:
sc_df.loc[(sc_df.SpinOff1MatchCnt > 1) & (sc_df.SpinOff1Match) & (sc_df.ev2.isna())]

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,...,Total_test,EndVal,EndValMax,EndValMatch,EndValMatchCnt,SpinOff1,Removal,SpinOff1Max,SpinOff1Match,SpinOff1MatchCnt


In [438]:
sc_df['SpinOff2'] = np.select([(sc_df.SpinOff1MatchCnt > 1) & (sc_df.SpinOff1Match)], [sc_df.ev2], None)
sc_df['SpinOff2'] = sc_df.SpinOff2.astype(float)

In [439]:
display_cols = ['Contestant', 'Spin1', 'Spin2', 'Total', 'ev1', 'Comments', 'Showcase', 'urlParam', 'ev2', 'ev3', 'ev4',
                'ContestantOrder', 'CommNums', 'Total_test', 'SpinOff1', 'Removal', 'SpinOff2']
sc_df.loc[~sc_df.SpinOff2.isna(), display_cols]

,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,SpinOff1,Removal,SpinOff2
45,Vanessa,10.0,85.0,95,95.0,C: Through to the Showcases,SS2,9763K,85.0,NaN,NaN,1.0,[],95,95.0,False,85.0
47,Ayesha,95.0,0.0,95,95.0,C:,SS2,9763K,45.0,NaN,NaN,3.0,[],95,95.0,False,45.0
729,Joven,20.0,65.0,85,85.0,C: Through to the Showcases,SS2,9531K,60.0,NaN,NaN,1.0,[],85,85.0,False,60.0
731,Kayla,40.0,45.0,85,85.0,C:,SS2,9531K,15.0,NaN,NaN,3.0,[],85,85.0,False,15.0
853,Gustave,95.0,0.0,95,20.0,C:,SS2,9482K,20.0,NaN,NaN,2.0,[],95,20.0,False,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23399,Beverly,90.0,0.0,90,75.0,C: Through to the Showcases,SS2,4783D,25.0,NaN,NaN,3.0,[],90,75.0,False,25.0
23838,Patricia,55.0,25.0,80,90.0,C: Through to the Showcases,SS2,4622D,95.0,NaN,NaN,1.0,[],80,90.0,False,95.0
23840,Mark,5.0,75.0,80,90.0,C:,SS2,4622D,70.0,NaN,NaN,3.0,[],80,90.0,False,70.0
24238,Randy,90.0,0.0,90,85.0,C: Through to the Showcases,SS1,3954D,45.0,NaN,NaN,2.0,[],90,85.0,False,45.0


#### SpinOff3

In [440]:
sc_df.head(3)

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,...,EndVal,EndValMax,EndValMatch,EndValMatchCnt,SpinOff1,Removal,SpinOff1Max,SpinOff1Match,SpinOff1MatchCnt,SpinOff2
0,1,"1,340",Maurice,85.0,0.0,85,NaN,C:,SS1,9782K,...,85,100,False,1,NaN,False,NaN,False,0,NaN
1,2,"8,531",Kim,100.0,0.0,100,50.0,C: Bonus Spin ►,SS1,9782K,...,100,100,True,1,NaN,False,NaN,False,0,NaN
2,3,"25,760",Sandra,20.0,50.0,70,NaN,C:,SS1,9782K,...,70,100,False,1,NaN,False,NaN,False,0,NaN


In [441]:
sc_df = get_tie_df(sc_df, 'SpinOff2')

In [442]:
sc_df.loc[(sc_df.SpinOff2MatchCnt.astype(float) > 1) & sc_df.SpinOff2Match, display_cols]

,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,SpinOff1,Removal,SpinOff2
13141,Mildred,35.0,60.0,95,30.0,C: Through to the Showcases,SS2,4801K,20.0,90.0,NaN,2.0,[],95,30.0,False,20.0
13142,Mccall,95.0,0.0,95,30.0,C:,SS2,4801K,20.0,85.0,NaN,3.0,[],95,30.0,False,20.0
13999,Daniel,95.0,0.0,95,50.0,C:,SS1,4494K,5.0,20.0,NaN,2.0,[],95,50.0,False,5.0
14000,Vito,80.0,15.0,95,50.0,C: Through...FINALLY!,SS1,4494K,5.0,80.0,NaN,3.0,[],95,50.0,False,5.0
16540,Daniel,5.0,80.0,85,10.0,C: Through to the Showcases,SS2,1051K,50.0,95.0,NaN,2.0,[],850,10.0,False,50.0
16541,Jaime,50.0,35.0,85,10.0,C:,SS2,1051K,50.0,10.0,NaN,3.0,[],850,10.0,False,50.0
16794,Trina,75.0,0.0,75,25.0,C: Through to the Showcases,SS2,0925K,65.0,95.0,NaN,1.0,[],75,25.0,False,65.0
16796,Mary,75.0,0.0,75,25.0,C:,SS2,0925K,65.0,45.0,NaN,3.0,[],75,25.0,False,65.0
17752,Rafael,85.0,0.0,85,40.0,C:,SSSO,0222K,10.0,30.0,NaN,2.0,[],85,40.0,False,10.0
17753,Melissa,85.0,0.0,85,40.0,C: to the showacse,SSSO,0222K,10.0,85.0,NaN,3.0,[],85,40.0,False,10.0


In [443]:
sc_df['SpinOff3'] = np.select([(sc_df.SpinOff2MatchCnt > 1) & (sc_df.SpinOff2Match)], [sc_df.ev3], None)
sc_df['SpinOff3'] = sc_df.SpinOff3.astype(float)

In [444]:
sc_df.loc[(sc_df.SpinOff2MatchCnt.astype(float) > 1) & sc_df.SpinOff2Match, display_cols + ['SpinOff3']]

,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,SpinOff1,Removal,SpinOff2,SpinOff3
13141,Mildred,35.0,60.0,95,30.0,C: Through to the Showcases,SS2,4801K,20.0,90.0,NaN,2.0,[],95,30.0,False,20.0,90.0
13142,Mccall,95.0,0.0,95,30.0,C:,SS2,4801K,20.0,85.0,NaN,3.0,[],95,30.0,False,20.0,85.0
13999,Daniel,95.0,0.0,95,50.0,C:,SS1,4494K,5.0,20.0,NaN,2.0,[],95,50.0,False,5.0,20.0
14000,Vito,80.0,15.0,95,50.0,C: Through...FINALLY!,SS1,4494K,5.0,80.0,NaN,3.0,[],95,50.0,False,5.0,80.0
16540,Daniel,5.0,80.0,85,10.0,C: Through to the Showcases,SS2,1051K,50.0,95.0,NaN,2.0,[],850,10.0,False,50.0,95.0
16541,Jaime,50.0,35.0,85,10.0,C:,SS2,1051K,50.0,10.0,NaN,3.0,[],850,10.0,False,50.0,10.0
16794,Trina,75.0,0.0,75,25.0,C: Through to the Showcases,SS2,0925K,65.0,95.0,NaN,1.0,[],75,25.0,False,65.0,95.0
16796,Mary,75.0,0.0,75,25.0,C:,SS2,0925K,65.0,45.0,NaN,3.0,[],75,25.0,False,65.0,45.0
17752,Rafael,85.0,0.0,85,40.0,C:,SSSO,0222K,10.0,30.0,NaN,2.0,[],85,40.0,False,10.0,30.0
17753,Melissa,85.0,0.0,85,40.0,C: to the showacse,SSSO,0222K,10.0,85.0,NaN,3.0,[],85,40.0,False,10.0,85.0


### Bonus Spin Value if spun 100
Beginning on December 12, 1978, hitting \$1.00 not only won the \$1,000, but also a bonus spin

In [445]:
new_base = sc_df.copy()

In [446]:
# Find first value of 100 and then take the next value as the Bonus
sc_df['Bonus'] = np.select([sc_df.Total == 100, sc_df.SpinOff1 == 100, sc_df.SpinOff2 == 100],
                          [sc_df.ev1, sc_df.ev2, sc_df.ev3], None)
sc_df.loc[sc_df.SpinOff2 == 100, display_cols + ['SpinOff3', 'Bonus']]

,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,ev2,ev3,ev4,ContestantOrder,CommNums,Total_test,SpinOff1,Removal,SpinOff2,SpinOff3,Bonus
2273,Anthony,60.0,10.0,70,100.0,"C: 1.00 1.00 25,000 Bonus",SS1,8942K,100.0,NaN,NaN,3.0,"[1.00, 1.00]",70,100.0,False,100.0,NaN,100.0
2345,Marilyn,80.0,0.0,80,90.0,C: 1.00 Bonus Spin ►,SS2,8911K,100.0,25.0,NaN,3.0,[1.00],80,90.0,False,100.0,NaN,25.0
15453,Christopher,45.0,45.0,90,55.0,C: bonus spin,SS1,1511K,100.0,95.0,NaN,1.0,[],900,55.0,False,100.0,NaN,95.0
16717,Beverly,100.0,0.0,100,100.0,"C: *1,000* BONUS ***10,000***",SS2,0972K,100.0,NaN,NaN,2.0,[],100,100.0,False,100.0,NaN,100.0
19446,Rodney,90.0,0.0,90,90.0,"C: *1,000* BONUS",SS2,8704D,100.0,10.0,NaN,1.0,[],90,90.0,False,100.0,NaN,10.0
22075,Jan,55.0,25.0,80,55.0,C: 1.00 Bonus Spin >,SS1,5265D,100.0,20.0,NaN,2.0,[1.00],80,55.0,False,100.0,NaN,20.0
22823,Stephen,50.0,0.0,50,45.0,C: 1.00 Bonus Spin >,SS1,4993D,100.0,60.0,NaN,3.0,[1.00],50,45.0,False,100.0,NaN,60.0


### Final Removal of Bad Data & Combine with Content

In [447]:
print(sc_df.shape)
sc_df = sc_df.loc[~sc_df.Removal].copy()
del sc_df['Removal']
sc_df.shape

(24933, 31)


(24898, 30)

In [448]:
sc_df = keep_only_with_three(sc_df)
sc_df.shape

3    0.996511
2    0.002767
1    0.000722
Name: SC_ind, dtype: float64

(24846, 30)

#### Winner Column

In [449]:
sc_df = get_tie_df(sc_df, 'SpinOff3')

In [459]:
sc_df['Winner'] = np.select([sc_df.SpinOff3Match, sc_df.SpinOff3Max.isna() & sc_df.SpinOff2Match,
                             sc_df.SpinOff2Max.isna() & sc_df.SpinOff1Match, sc_df.SpinOff1Max.isna() & sc_df.EndValMatch],
                            [True, True,  True, True], False)
sc_df.groupby(['Showcase', 'urlParam']).Winner.sum().reset_index().Winner.value_counts()

1    8282
Name: Winner, dtype: int64

In [460]:
sc_df.columns

Index(['SC_ind', 'PrizeValue', 'Contestant', 'Spin1', 'Spin2', 'Total', 'ev1',
       'Comments', 'Showcase', 'urlParam', 'ev2', 'ev3', 'ev4',
       'ContestantOrder', 'CommNums', 'Total_test', 'EndVal', 'EndValMax',
       'EndValMatch', 'EndValMatchCnt', 'SpinOff1', 'SpinOff1Max',
       'SpinOff1Match', 'SpinOff1MatchCnt', 'SpinOff2', 'SpinOff2Max',
       'SpinOff2Match', 'SpinOff2MatchCnt', 'SpinOff3', 'Bonus', 'SpinOff3Max',
       'SpinOff3Match', 'SpinOff3MatchCnt', 'Winner'],
      dtype='object')

#### Final Merge

In [461]:
display(sc_df.head(3))
display(main_df.head(3))

,SC_ind,PrizeValue,Contestant,Spin1,Spin2,Total,ev1,Comments,Showcase,urlParam,...,SpinOff2,SpinOff2Max,SpinOff2Match,SpinOff2MatchCnt,SpinOff3,Bonus,SpinOff3Max,SpinOff3Match,SpinOff3MatchCnt,Winner
0,1,"1,340",Maurice,85.0,0.0,85,NaN,C:,SS1,9782K,...,NaN,NaN,False,0,NaN,None,NaN,False,0,False
1,2,"8,531",Kim,100.0,0.0,100,50.0,C: Bonus Spin ►,SS1,9782K,...,NaN,NaN,False,0,NaN,50.0,NaN,False,0,True
2,3,"25,760",Sandra,20.0,50.0,70,NaN,C:,SS1,9782K,...,NaN,NaN,False,0,NaN,None,NaN,False,0,False


,url,episodeID,errorCode,footer,urlParam,EpsCategory,airDate
0,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29
10,https://tpirepguide.com/?p=18045,9781K,0,"March 28th, 2022 | Category: Carey Eps",18045,Carey Eps,2022-03-28
11,https://tpirepguide.com/?p=18018,9775K,0,"March 25th, 2022 | Category: Carey Eps",18018,Carey Eps,2022-03-25


In [462]:
final_df = main_df.merge(sc_df, how='left', left_on=['episodeID'], right_on=['urlParam'], suffixes=("", "_sc"))
final_df.head(3)

,url,episodeID,errorCode,footer,urlParam,EpsCategory,airDate,SC_ind,PrizeValue,Contestant,...,SpinOff2,SpinOff2Max,SpinOff2Match,SpinOff2MatchCnt,SpinOff3,Bonus,SpinOff3Max,SpinOff3Match,SpinOff3MatchCnt,Winner
0,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29,1.0,"1,340",Maurice,...,NaN,NaN,False,0.0,NaN,None,NaN,False,0.0,False
1,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29,2.0,"8,531",Kim,...,NaN,NaN,False,0.0,NaN,50.0,NaN,False,0.0,True
2,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29,3.0,"25,760",Sandra,...,NaN,NaN,False,0.0,NaN,None,NaN,False,0.0,False


In [463]:
final_df.columns

Index(['url', 'episodeID', 'errorCode', 'footer', 'urlParam', 'EpsCategory',
       'airDate', 'SC_ind', 'PrizeValue', 'Contestant', 'Spin1', 'Spin2',
       'Total', 'ev1', 'Comments', 'Showcase', 'urlParam_sc', 'ev2', 'ev3',
       'ev4', 'ContestantOrder', 'CommNums', 'Total_test', 'EndVal',
       'EndValMax', 'EndValMatch', 'EndValMatchCnt', 'SpinOff1', 'SpinOff1Max',
       'SpinOff1Match', 'SpinOff1MatchCnt', 'SpinOff2', 'SpinOff2Max',
       'SpinOff2Match', 'SpinOff2MatchCnt', 'SpinOff3', 'Bonus', 'SpinOff3Max',
       'SpinOff3Match', 'SpinOff3MatchCnt', 'Winner'],
      dtype='object')

In [464]:
final_df.loc[:, ['url', 'episodeID', 'errorCode', 'footer', 'urlParam', 'EpsCategory',
       'airDate', 'SC_ind', 'PrizeValue', 'Contestant', 'Spin1', 'Spin2',
       'Total', 'Comments', 'Showcase', 'ev1', 'ev2', 'ev3',
       'ev4', 'ContestantOrder', 'CommNums', 'Total_test', 'EndVal',
       'EndValMax', 'EndValMatch', 'EndValMatchCnt', 'SpinOff1', 'SpinOff1Max',
       'SpinOff1Match', 'SpinOff1MatchCnt', 'SpinOff2', 'SpinOff2Max',
       'SpinOff2Match', 'SpinOff2MatchCnt', 'SpinOff3', 'SpinOff3Max',
       'SpinOff3Match', 'SpinOff3MatchCnt', 'Bonus', 'Winner']].to_csv(r"FullContent.csv", header=True, index=False)

In [ ]:
main_df.loc[main_df.episodeID == '4675K']

### Create CrossTab w/ all 3 contestants in 1 Row

In [3]:
file_path = r"FullContent.csv"
df = pd.read_csv(file_path, parse_dates=['airDate']).drop_duplicates()
df.head(3)

,url,episodeID,errorCode,footer,urlParam,EpsCategory,airDate,SC_ind,PrizeValue,Contestant,...,SpinOff2,SpinOff2Max,SpinOff2Match,SpinOff2MatchCnt,SpinOff3,SpinOff3Max,SpinOff3Match,SpinOff3MatchCnt,Bonus,Winner
0,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29,1.0,"1,340",Maurice,...,NaN,NaN,False,0.0,NaN,NaN,False,0.0,NaN,False
1,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29,2.0,"8,531",Kim,...,NaN,NaN,False,0.0,NaN,NaN,False,0.0,50.0,True
2,https://tpirepguide.com/?p=18052,9782K,0,"March 29th, 2022 | Category: Carey Eps",18052,Carey Eps,2022-03-29,3.0,"25,760",Sandra,...,NaN,NaN,False,0.0,NaN,NaN,False,0.0,NaN,False


In [4]:
df = df.loc[~df.ContestantOrder.isna()].copy()
df.shape

(24864, 40)

In [5]:
remove_cols = ['ev1', 'ev2', 'ev3', 'ev4', 'SC_ind', 'footer', 'Comments', 'Total_test', 'CommNums']
for c in remove_cols:
    del df[c]
df['Spin1'] = df['Spin1'].astype(np.int64)
df['Spin2'] = df['Spin2'].astype(np.int64)
df['Total'] = df['Total'].astype(np.int64)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24864 entries, 0 to 26790
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   url               24864 non-null  object        
 1   episodeID         24864 non-null  object        
 2   errorCode         24864 non-null  int64         
 3   urlParam          24864 non-null  int64         
 4   EpsCategory       24864 non-null  object        
 5   airDate           24864 non-null  datetime64[ns]
 6   PrizeValue        24864 non-null  object        
 7   Contestant        24864 non-null  object        
 8   Spin1             24864 non-null  int64         
 9   Spin2             24864 non-null  int64         
 10  Total             24864 non-null  int64         
 11  Showcase          24864 non-null  object        
 12  ContestantOrder   24864 non-null  float64       
 13  EndVal            24864 non-null  float64       
 14  EndValMax         2486

In [6]:
other_cols = ['urlParam', 'Showcase', 'PrizeValue', 'Contestant', 'Spin1', 'Spin2', 'Total',
              'EndValMatch', 'SpinOff1', 'SpinOff2', 'SpinOff3', 'Bonus', 'Winner']
c1_df = df.loc[(df.ContestantOrder == 1)].copy()
c2_df = df.loc[(df.ContestantOrder == 2), other_cols].copy()
c3_df = df.loc[(df.ContestantOrder == 3), other_cols].copy()
print(c1_df.shape, c2_df.shape, c3_df.shape)

(8288, 31) (8288, 13) (8288, 13)


Merge

In [7]:
ct_df = c1_df.merge(c2_df, how='left', on=['urlParam', 'Showcase'], suffixes=("", '_C2'))
ct_df = ct_df.merge(c3_df, how='left', on=['urlParam', 'Showcase'], suffixes=("", '_C3'))
ct_df.shape

(8288, 53)

In [8]:
ct_df.columns

Index(['url', 'episodeID', 'errorCode', 'urlParam', 'EpsCategory', 'airDate',
       'PrizeValue', 'Contestant', 'Spin1', 'Spin2', 'Total', 'Showcase',
       'ContestantOrder', 'EndVal', 'EndValMax', 'EndValMatch',
       'EndValMatchCnt', 'SpinOff1', 'SpinOff1Max', 'SpinOff1Match',
       'SpinOff1MatchCnt', 'SpinOff2', 'SpinOff2Max', 'SpinOff2Match',
       'SpinOff2MatchCnt', 'SpinOff3', 'SpinOff3Max', 'SpinOff3Match',
       'SpinOff3MatchCnt', 'Bonus', 'Winner', 'PrizeValue_C2', 'Contestant_C2',
       'Spin1_C2', 'Spin2_C2', 'Total_C2', 'EndValMatch_C2', 'SpinOff1_C2',
       'SpinOff2_C2', 'SpinOff3_C2', 'Bonus_C2', 'Winner_C2', 'PrizeValue_C3',
       'Contestant_C3', 'Spin1_C3', 'Spin2_C3', 'Total_C3', 'EndValMatch_C3',
       'SpinOff1_C3', 'SpinOff2_C3', 'SpinOff3_C3', 'Bonus_C3', 'Winner_C3'],
      dtype='object')

In [9]:
ct_df['Winner'] = ct_df['Winner'].astype(bool)
ct_df['Winner_C2'] = ct_df['Winner_C2'].astype(bool)
ct_df['Winner_C3'] = ct_df['Winner_C3'].astype(bool)
ct_df['winnerNum'] = np.select([ct_df.Winner, ct_df.Winner_C2, ct_df.Winner_C3], [1, 2, 3], None)
ct_df.winnerNum.value_counts()

3    2955
2    2831
1    2502
Name: winnerNum, dtype: int64

In [13]:
# Add Result for Flow Diagram later
# C1 --> Over or Stays
ct_df['C1_Result'] = np.select([ct_df.Total > 100], ['Over'], 'Stay')
ct_df.groupby(['C1_Result'])['Total'].agg(['count', 'min', 'max']).reset_index()

,C1_Result,count,min,max
0,Over,1711,105,165
1,Stay,6577,10,100


In [14]:
# C2 --> Over, Ties, Stays, Under
ct_df['C2_Result'] = np.select([ct_df.Total_C2 > 100, ct_df.C1_Result == 'Over',
                                ct_df.Total_C2 < ct_df.Total, ct_df.Total_C2 == ct_df.Total],
                              ['Over', 'Stay', 'Under', 'Tie'], 'Stay')

In [15]:
ct_df.groupby(['C1_Result', 'C2_Result'])[['Total', 'Total_C2']].agg(['count', 'min', 'max']).reset_index()

C1_Result C2_Result Total           Total_C2          
                      count  min  max    count  min  max
0      Over      Over   272  105  160      272  105  165
1      Over      Stay  1439  105  165     1439   10  100
2      Stay      Over  1973   10  100     1973  105  195
3      Stay      Stay  2573   10   95     2573   30  100
4      Stay       Tie   528   25  100      528   25  100
5      Stay     Under  1503   20  100     1503   10   95

In [17]:
# C2 --> Over, Ties, Stays, Under
ct_df['C3_Result'] = np.select([ct_df.Total_C3 > 100, (ct_df.C1_Result == 'Over') & (ct_df.C2_Result == 'Over'),
                                (ct_df.C2_Result.isin(['Stay', 'Tie'])) & (ct_df.Total_C3 < ct_df.Total_C2),
                                (ct_df.C2_Result.isin(['Stay', 'Tie'])) & (ct_df.Total_C3 > ct_df.Total_C2),
                                (ct_df.C2_Result.isin(['Stay', 'Tie'])) & (ct_df.Total_C3 == ct_df.Total_C2),
                                (ct_df.C2_Result.isin(['Over', 'Under'])) & (ct_df.Total_C3 < ct_df.Total),
                                (ct_df.C2_Result.isin(['Over', 'Under'])) & (ct_df.Total_C3 > ct_df.Total),
                                (ct_df.C2_Result.isin(['Over', 'Under'])) & (ct_df.Total_C3 == ct_df.Total)],
                               ['Over', 'Stay', 'Under', 'Stay', 'Tie', 'Under', 'Stay', 'Tie'], 'Unknown')

In [18]:
ct_df.groupby(['C1_Result', 'C2_Result', 'C3_Result'])[['Total', 'Total_C2',
                                                        'Total_C3']].agg(['count', 'min', 'max']).reset_index()

C1_Result C2_Result C3_Result Total           Total_C2           Total_C3  \
                                 count  min  max    count  min  max    count   
0       Over      Over      Stay   272  105  160      272  105  165      272   
1       Over      Stay      Over   395  105  165      395   30  100      395   
2       Over      Stay      Stay   597  105  165      597   10   95      597   
3       Over      Stay       Tie   108  105  165      108   40  100      108   
4       Over      Stay     Under   339  105  160      339   35  100      339   
5       Stay      Over      Over   645   40  100      645  105  190      645   
6       Stay      Over      Stay   608   10   95      608  105  190      608   
7       Stay      Over       Tie   162   50  100      162  105  180      162   
8       Stay      Over     Under   558   15  100      558  105  195      558   
9       Stay      Stay      Over   919   10   95      919   50  100      919   
10      Stay      Stay      Stay   598   10   90      598   30   95      598   
11      Stay      Stay       Tie   220   10   95      220   30  100      220   
12      Stay      Stay     Under   836   10   95      836   55  100      836   
13      Stay       Tie      Over   169   50  100      169   50  100      169   
14      Stay       Tie      Stay   164   25   95      164   25   95      164   
15      Stay       Tie       Tie    53   50  100       53   50  100       53   
16      Stay       Tie     Under   142   60  100      142   60  100      142   
17      Stay     Under      Over   530   30  100      530   15   95      530   
18      Stay     Under      Stay   408   20   95      408   10   90      408   
19      Stay     Under       Tie   119   45  100      119   15   95      119   
20      Stay     Under     Under   446   35  100      446   10   95      446   

              
    min  max  
0     5  100  
1   105  190  
2    25  100  
3    40  100  
4    10   95  
5   105  195  
6    20  100  
7    50  100  
8    10   95  
9   105  195  
10   45  100  
11   30  100  
12   10   95  
13  105  185  
14   35  100  
15   50  100  
16   10   95  
17  105  190  
18   40  100  
19   45  100  
20   10   95

In [19]:
ct_df.groupby(['C1_Result', 'C2_Result', 'C3_Result', 'winnerNum'])[['Total', 'Total_C2',
                                                        'Total_C3']].agg(['count', 'min', 'max']).reset_index()

C1_Result C2_Result C3_Result winnerNum Total           Total_C2            \
                                           count  min  max    count  min  max   
0       Over      Over      Stay         3   272  105  160      272  105  165   
1       Over      Stay      Over         2   395  105  165      395   30  100   
2       Over      Stay      Stay         3   597  105  165      597   10   95   
3       Over      Stay       Tie         2    54  105  165       54   40  100   
4       Over      Stay       Tie         3    54  105  155       54   45  100   
5       Over      Stay     Under         2   339  105  160      339   35  100   
6       Stay      Over      Over         1   645   40  100      645  105  190   
7       Stay      Over      Stay         3   608   10   95      608  105  190   
8       Stay      Over       Tie         1    84   50  100       84  105  180   
9       Stay      Over       Tie         3    78   50  100       78  105  175   
10      Stay      Over     Under         1   558   15  100      558  105  195   
11      Stay      Stay      Over         2   919   10   95      919   50  100   
12      Stay      Stay      Stay         3   598   10   90      598   30   95   
13      Stay      Stay       Tie         2   110   10   95      110   30  100   
14      Stay      Stay       Tie         3   110   10   95      110   45  100   
15      Stay      Stay     Under         2   836   10   95      836   55  100   
16      Stay       Tie      Over         1    93   50  100       93   50  100   
17      Stay       Tie      Over         2    76   60  100       76   60  100   
18      Stay       Tie      Stay         3   164   25   95      164   25   95   
19      Stay       Tie       Tie         1    22   50  100       22   50  100   
20      Stay       Tie       Tie         2    20   50  100       20   50  100   
21      Stay       Tie       Tie         3    11   65  100       11   65  100   
22      Stay       Tie     Under         1    60   60  100       60   60  100   
23      Stay       Tie     Under         2    82   60  100       82   60  100   
24      Stay     Under      Over         1   530   30  100      530   15   95   
25      Stay     Under      Stay         3   408   20   95      408   10   90   
26      Stay     Under       Tie         1    64   50  100       64   15   95   
27      Stay     Under       Tie         3    55   45  100       55   25   95   
28      Stay     Under     Under         1   446   35  100      446   10   95   

   Total_C3            
      count  min  max  
0       272    5  100  
1       395  105  190  
2       597   25  100  
3        54   40  100  
4        54   45  100  
5       339   10   95  
6       645  105  195  
7       608   20  100  
8        84   50  100  
9        78   50  100  
10      558   10   95  
11      919  105  195  
12      598   45  100  
13      110   30  100  
14      110   45  100  
15      836   10   95  
16       93  105  175  
17       76  105  185  
18      164   35  100  
19       22   50  100  
20       20   50  100  
21       11   65  100  
22       60   15   95  
23       82   10   90  
24      530  105  190  
25      408   40  100  
26       64   50  100  
27       55   45  100  
28      446   10   95

In [20]:
ct_df.to_csv(r"OneRowData.csv", header=True, index=False)

In [478]:
year_sum = final_df.groupby(['urlParam', final_df.airDate.dt.year]).ContestantOrder.count().reset_index()
year_sum['Found'] = np.select([year_sum.ContestantOrder > 0], [1], 0)
year_sum = year_sum.groupby('airDate').Found.agg(['sum', 'count']).reset_index()

In [479]:
year_sum

,airDate,sum,count
0,1972,0,84
1,1973,0,80
2,1974,0,30
3,1975,6,35
4,1976,17,22
5,1977,29,31
6,1978,16,18
7,1979,20,20
8,1980,33,38
9,1981,28,30
